In [ ]:
import pandas as pd
import numpy as np 
import geopandas as gpd
import os 

from generate_random_location import *

In [ ]:
hydrogen_car_all = pd.read_excel('./data/2022년 7월말 수소차 구별 현황.xlsx')
charge_data = gpd.read_file('./data/대전 수소충전소 현황/대전 수소충전소 현황.shp')

In [ ]:
hydrogen_car_all.head()

---
### 데이터 생성 

- 대전 수소차 random 위치 생성 코드

In [ ]:
hydrogen_car_donggu = Generate_taxi_random_location('동구, 대전, 대한민국', 177)
hydrogen_car_jonggu = Generate_taxi_random_location('중구, 대전, 대한민국', 171)
hydrogen_car_sugu = Generate_taxi_random_location('서구, 대전, 대한민국', 375)
hydrogen_car_usunggu = Generate_taxi_random_location('유성구, 대전, 대한민국', 341)
hydrogen_car_daeduckgu = Generate_taxi_random_location('대덕구, 대전, 대한민국', 127)

###
hydrogen_car_donggu = pd.DataFrame(hydrogen_car_donggu, columns=['geometry'])
hydrogen_car_donggu['구'] = '동구'
hydrogen_car_jonggu = pd.DataFrame(hydrogen_car_jonggu, columns=['geometry'])
hydrogen_car_jonggu['구'] = '중구'
hydrogen_car_sugu = pd.DataFrame(hydrogen_car_sugu, columns=['geometry'])
hydrogen_car_sugu['구'] = '서구'
hydrogen_car_usunggu = pd.DataFrame(hydrogen_car_usunggu, columns=['geometry'])
hydrogen_car_usunggu['구'] = '유성구'
hydrogen_car_daeduckgu = pd.DataFrame(hydrogen_car_daeduckgu, columns=['geometry'])
hydrogen_car_daeduckgu['구'] = '대덕구'

###
hydrogen_car_geometry = pd.concat([hydrogen_car_donggu, hydrogen_car_jonggu, hydrogen_car_sugu, hydrogen_car_usunggu, hydrogen_car_daeduckgu])

### 다시 만들기 시간 오래 걸리니까 저장
hydrogen_car_geometry['lon'] = [i.x for i in hydrogen_car_geometry['geometry']]
hydrogen_car_geometry['lat'] = [i.y for i in hydrogen_car_geometry['geometry']]

hydrogen_car_geometry.to_csv('hydrogen_car_geometry.csv', index=False)

sample_car = hydrogen_car_geometry.sample(n=79) 

In [ ]:
print('시뮬레이션 차량 대수 : ',len(sample_car))

- 충전 차량 요청 시간
    - 가정: 교통량이 많은 시간에 충전을 하려는 차량도 많을 것이다.
        - 교통량이 많은 시간 오전 첨두시(7시~9시), 오후 첨두시(18~20시)
        - 오전 첨두시에는 출근 길이기 때문에 오후 첨두시에 충전하는 사람이 많을 것으로 예측
    - 가정을 기반하여, 시간을 랜덤 부여 
        - 오전 첨두시 30%, 오후 첨두시 60%, 그 외 시간 10% 

In [19]:
### 차량 충전 하러 가는 시간 
AM_time_list = list(range(420, 540))   # 24대
PM_time_list = list(range(1080, 1200)) # 47대
etc_time_list = list(range(240, 1400)) # 8대 

AM_random_time_list = np.random.choice(AM_time_list, size=16, replace=True)
PM_random_time_list = np.random.choice(PM_time_list, size=32, replace=True)
etc_random_time_list = np.random.choice(etc_time_list, size=5, replace=True)

time_list = []

time_list.extend(AM_random_time_list)
time_list.extend(PM_random_time_list)
time_list.extend(etc_random_time_list)

sample_car['charge_time'] = time_list

sample_car.to_csv('hydrogen_car_sample.csv', index=False)

```Python
### 대전 수소차 충전소 시각화
import folium
from folium import Marker
from folium import Circle
import osmnx as ox 

###대전광역시_법정경계 Data
place = "대전, 대한민국"

places = ox.geocode_to_gdf([place])
places = ox.project_gdf(places)


#lat, lon
latitude = places.lat.values[0]; longitude = places.lon.values[0]
#기본 지도 정의
m = folium.Map(location=[latitude, longitude],
               zoom_start=11)

#법정경계 표시
folium.Choropleth(geo_data=places.geometry,
                  fill_color="white",
                  ).add_to(m)

for _, row in sample_car.iterrows():
    Circle(location = [row['lat'], row['lon']],
           icon=folium.Icon(color='blue',icon='star'),
           radius=100
          ).add_to(m)
    
    
for _, row in charge_data.iterrows():
    Marker(location = [row['위도'], row['경도']],
           popup=row['이름'],
           icon=folium.Icon(color='red',icon='star')
          ).add_to(m)

m
```

- 영업소 정보 (영업 시간, 휴계시간, 충전 가능 대수)
    - 학하수소충전소 : 9~20, 12~13, 1
    - 신대수소충전소 : 4~24, x, 2
    - 낭월수소충전소 : 4~24, x, 2
    - 대전유성 : 8~20, x, 2
    - 자운대 :8~20, x, 1
    - 중도 : 8~20, x, 1
    - 신탄진 : 8~22, x, 1

In [ ]:
charge_data['capacity'] = [1,2,2,2,1,1,1]
charge_data['business_hours'] = [[540, 1200],[240,1440],[240,1440],[480,1200],[480,1200],[480,1200],[480,1320]]
charge_data['break_time'] = [[720, 780], [0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]

charge_data.to_csv('./data/prepared_data/charging_station.csv', index=False)

- 완충시 주행거리 고려하여, 전체 차량의 1/15만 하루에 충전한다고 가정하여, 소수차 전체 차량의 1/15만 생성

In [ ]:
hydrogen_car_geometry = pd.read_csv('./data/prepared_data/hydrogen_car_geometry.csv')
sample_car = hydrogen_car_geometry.sample(frac=1/15) 

sample_car.to_csv('hydrogen_car_sample.csv', index=False)

---

### Simulation

In [1]:
import pandas as pd
import numpy as np 
from shapely.geometry import Point
from dispatch_ortools import *
from osrm_api import *
import json
import pickle

In [2]:
# 주유소 추가 함수
def add_station_func(charge_station):
    add_station = pd.DataFrame([['sk엔크린_KH에너지직영_태양주유소','-',36.355570,127.436234,'-', 2, [480,1440], [0,0]],
                ['중도가스_서대전충전소','-',36.326030,127.403108,'-', 2, [480,1440], [0,0]]], 
                columns=['이름', '주소', '위도', '경도', 'geometry', 'capacity', 'business_hours','break_time'])

    charge_station = pd.concat([charge_station,add_station])
    charge_station = charge_station.reset_index(drop=True)
    return charge_station

### 시나리오 선택
# 1 : 기본 시나리오 (충전소 : 7개, 차량 : 79개)
# 2 : 기본 시나리오 + 충전소 추가 (충전소 : 9개, 차량 79개)
# 3 : 기본 시나리오 + 차량 추가 (충전소 : 7개, 차량 132개)  2023 증가 비율로 추가 반영 
# 4 : 기본 시나리오 + 차량 추가 + 충전소 추가 (충전소 9개, 차량 132개)
def select_scenario(mode='scen_1'):
    if mode == 'scen_1':
        charge_station = pd.read_csv('./data/prepared_data/charging_station.csv')
        hydrogen_car_sample = pd.read_csv('./data/prepared_data/hydrogen_car_sample_79.csv')
        
        charge_station['business_hours'] = [eval(i) for i in charge_station['business_hours']]
    elif mode == 'scen_2':
        charge_station = pd.read_csv('./data/prepared_data/charging_station.csv')
        hydrogen_car_sample = pd.read_csv('./data/prepared_data/hydrogen_car_sample_79.csv')        
        
        charge_station['business_hours'] = [eval(i) for i in charge_station['business_hours']] 
        charge_station = add_station_func(charge_station)

    elif mode == 'scen_3':
        charge_station = pd.read_csv('./data/prepared_data/charging_station.csv')
        hydrogen_car_sample = pd.read_csv('./data/prepared_data/hydrogen_car_sample_79.csv')
        extra_car_sample = pd.read_csv('./data/prepared_data/hydrogen_car_sample_extra_53.csv')
        
        charge_station['business_hours'] = [eval(i) for i in charge_station['business_hours']]
        hydrogen_car_sample = pd.concat([hydrogen_car_sample, extra_car_sample])
        hydrogen_car_sample = hydrogen_car_sample.reset_index(drop=True)
    elif mode == 'scen_4':
        charge_station = pd.read_csv('./data/prepared_data/charging_station.csv')
        hydrogen_car_sample = pd.read_csv('./data/prepared_data/hydrogen_car_sample_79.csv')
        extra_car_sample = pd.read_csv('./data/prepared_data/hydrogen_car_sample_extra_53.csv')
        
        charge_station['business_hours'] = [eval(i) for i in charge_station['business_hours']]
        charge_station = add_station_func(charge_station)
        
        hydrogen_car_sample = pd.concat([hydrogen_car_sample, extra_car_sample])
        hydrogen_car_sample = hydrogen_car_sample.reset_index(drop=True)
        
    return charge_station, hydrogen_car_sample

In [3]:
def data_prepared(charge_station, hydrogen_car_sample):
    ###
    hydrogen_car_sample = hydrogen_car_sample.sort_values('charge_time')

    charge_station['geometry'] = [Point(i[0],i[1]) for i in charge_station[['경도','위도']].values]
    hydrogen_car_sample['geometry'] = [Point(i[0],i[1]) for i in hydrogen_car_sample[['lon','lat']].values]

    charge_station['car_count'] = 0 
    charge_station['car_accumulate_count'] = 0 
    charge_station['station_id'] = [f'station_{i}' for i in range(len(charge_station))]
    charge_station['charge_wait_time'] = [[0,0] if i == 2 else [0] for i in charge_station['capacity']]
    charge_station['open_hour'] = [int(i[0]) for i in charge_station['business_hours']]
    charge_station['close_hour'] = [int(i[1]) for i in charge_station['business_hours']]
    charge_station['car_arrival_time'] = [[] for i in range(len(charge_station))]
    return charge_station, hydrogen_car_sample

In [4]:
def simulation(charge_station,hydrogen_car_sample):
    ### 차량 정보
    # 충전소까지 가는 거리, 걸리는 시간
    to_station_time = []
    to_station_distance = []
    # 차량 처리 완료
    vehicle_data = []

    for i in range(300, 1440): 
        ### 충전하러 가는 차
        # 가장 가까운 충전소로 가는 가정 (현재 충전소의 현황을 모르기 때문에 이렇게 가정함)
        charging_car = hydrogen_car_sample.loc[hydrogen_car_sample['charge_time'] == i]
        
        # 이용 가능한 충전소
        available_charge_station = charge_station.loc[(charge_station['open_hour'] <= i) & (charge_station['close_hour'] >= i)]
        
        if len(charging_car) > 0:
            charging_car_iloc, station_iloc = main_dispatch_order(charging_car, available_charge_station, i)
            
            station_trans_dict = {data[1]['station_id']:idx for idx, data in enumerate(available_charge_station.iterrows())} 
            station_iloc = [station_trans_dict[i] for i in station_iloc]
            
            charging_car_sub = charging_car.iloc[charging_car_iloc]
            charge_station_sub = available_charge_station.iloc[station_iloc]
            
            ###
            all_steps = [get_res([O.x,O.y,D.x,D.y]) for O,D in zip(charging_car_sub['geometry'], charge_station_sub['geometry'])]
            
            all_routes = [get_total_route(step) for step in all_steps]
            all_timestamps = [get_total_timestamp(step) for step in all_steps]
            all_durations = [get_duration(step)/60 for step in all_steps]  # 시간이 초 단위이기 때문에 60을 나눠서 분으로 변경 
            all_distances = [round(get_distance(step)) for step in all_steps]     # 거리 단위 meter 
            ###        
            all_timestamps = [list(np.array(tstamp) + np.array(i)) for tstamp in all_timestamps]            
            
            charging_car_sub['route'] = all_routes
            charging_car_sub['timestamp'] = all_timestamps
            charging_car_sub['duration'] = all_durations
            charging_car_sub['distance'] = all_distances
            
            vehicle_data.append(charging_car_sub)
            to_station_time.append(all_durations)
            to_station_distance.append(all_distances)
            ###
            # station 정보 업데이트 
            for idx, time in zip(charge_station_sub.index.tolist(), charging_car_sub['timestamp'].values):
                # 차량 충전소 도착 시간
                car_arrival_time_list = charge_station['car_arrival_time'].iloc[idx]
                car_arrival_time_list.append(time[-1])
                charge_station.at[idx,'car_arrival_time'] = car_arrival_time_list
                # 누적 차량 수
                car_accumulate_count = charge_station['car_accumulate_count'].iloc[idx]
                car_accumulate_count += 1
                charge_station.at[idx,'car_accumulate_count'] = car_accumulate_count
                
    return charge_station, vehicle_data, to_station_time, to_station_distance

In [5]:
mode = 'scen_4'

charge_station, hydrogen_car_sample = select_scenario(mode)
charge_station, hydrogen_car_sample = data_prepared(charge_station, hydrogen_car_sample)
charge_station, vehicle_data, to_station_time, to_station_distance = simulation(charge_station, hydrogen_car_sample)

vehicle = pd.concat(vehicle_data).reset_index(drop=True)

- 충전소 대기시간 

In [6]:
all_wait_time_list = []
all_car_charge_time = []


for i in charge_station.iterrows():
    # 차량 충전소 waiting time 
    wait_time_list = []
    # 충전 시작 시간
    car_charge_time = []

    if i[1]['capacity'] == 1:
        wait_time = i[1]['charge_wait_time'][0]
        arrival_time = i[1]['car_arrival_time'].copy()
        target_arrival_time = float('inf')
        
        for j in i[1]['car_arrival_time']:
            target_arrival_time = min(arrival_time)
            arrival_time.remove(target_arrival_time)

            if (wait_time == 0) | (wait_time <= target_arrival_time): 
                
                wait_time_list.append(0)
                
                wait_time = target_arrival_time+12
                
                car_charge_time.append(target_arrival_time)
                
            elif wait_time > target_arrival_time:    
                        
                change_charge_time = wait_time
                
                wait_time_list.append(wait_time-target_arrival_time)
                
                wait_time = change_charge_time+12

                car_charge_time.append(change_charge_time)
                
    if i[1]['capacity'] == 2:
        wait_time_1 = i[1]['charge_wait_time'][0]
        wait_time_2 = i[1]['charge_wait_time'][1]
        arrival_time = i[1]['car_arrival_time'].copy()
        target_arrival_time = float('inf')
            
        for j in i[1]['car_arrival_time']:
            
            wait_time = min([wait_time_1, wait_time_2])
            wait_min_idx = [wait_time_1, wait_time_2].index(min([wait_time_1, wait_time_2]))
            
            target_arrival_time = min(arrival_time)
            arrival_time.remove(target_arrival_time)    
            
            if (wait_time == 0) | (wait_time <= target_arrival_time): 
                
                wait_time_list.append(0)
                
                if wait_min_idx == 0:
                    wait_time_1 = target_arrival_time+12
                elif wait_min_idx == 1:
                    wait_time_2 = target_arrival_time+12
                
                car_charge_time.append(target_arrival_time)
                
            elif wait_time > target_arrival_time:    
                        
                change_charge_time = wait_time
                
                wait_time_list.append(wait_time-target_arrival_time)
                
                if wait_min_idx == 0:
                    wait_time_1 = change_charge_time+12
                elif wait_min_idx == 1:
                    wait_time_2 = change_charge_time+12

                car_charge_time.append(change_charge_time)
                
    all_wait_time_list.append(wait_time_list)
    all_car_charge_time.append(car_charge_time)

In [7]:
trips = [{'route':i[1]['route'], 'timestamp':i[1]['timestamp']} for i in vehicle.iterrows()]
station = [{'id':i[1]['station_id'], 'loc':[i[1]['경도'],i[1]['위도']]} for i in charge_station.iterrows()]

with open(f'./data/result_data/hydrogen_car_trips_{mode}.json', 'w') as f:
    json.dump(trips,f)
    
with open(f'./data/result_data/station_location_{mode}.json', 'w') as f:
    json.dump(station,f)


# 전체 데이터 저장    
all_data = [all_wait_time_list ,to_station_time,to_station_distance, vehicle_data, charge_station]

with open(f'./data/result_data/all_data_{mode}.pickle', 'wb') as f:
    pickle.dump(all_data, f)

---